### Read Data into Pandas Data Frames

In [2]:
trainFile = "adult_train.csv" #Change it to your local directory
testFile = "adult_test.csv"

import pandas as pd
trainData = pd.read_csv(trainFile, sep=",", header=0)
testData = pd.read_csv(testFile, sep=",", header=0)
trainData.head()

,age,workclass,fnlwgt,education,education_num,marital-status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Get X and Y from Training Data

In [22]:
X1 = trainData.iloc[:, 0:14]
X2 = testData.iloc[:, 0:14]
Y1 = trainData.iloc[:, 14]
Y2 = testData.iloc[:, 14]
frames = [X1, X2]
X = pd.concat(frames)

import category_encoders as ce
le =  ce.OneHotEncoder(return_df=False,handle_missing=False,handle_unknown="ignore")
X_encoded = le.fit_transform(X)
X_encoded_train = X_encoded[0:X1.shape[0], :]
X_encoded_test = X_encoded[X1.shape[0]:X_encoded.shape[0], :]

### Section 2. Train the Random Forest Model

In [5]:
nTrees = 100
max_depth = 5
min_node_size = 5
verbose = 0

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=nTrees, max_depth=max_depth, random_state=0, verbose=verbose, min_samples_leaf=min_node_size)
clf.fit(X_encoded_train, Y1)
print(clf.feature_importances_)

[8.45965433e-02 7.20688576e-05 1.10168581e-03 7.46543676e-04
 1.33513848e-04 3.83428655e-04 1.01425311e-03 3.50299037e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 8.79059825e-04
 2.75225977e-02 1.19961612e-02 1.89666472e-03 1.58433586e-02
 1.78567979e-04 3.55256721e-04 8.66128502e-05 6.13527237e-05
 1.29263193e-03 2.44223206e-03 8.30664122e-03 1.30000061e-04
 1.03505638e-03 0.00000000e+00 0.00000000e+00 4.23358115e-05
 0.00000000e+00 7.25599956e-02 9.54802375e-02 1.45652824e-01
 1.24680922e-02 1.55132858e-04 7.30476903e-04 7.39613800e-05
 1.21532931e-04 0.00000000e+00 8.83823508e-04 2.32729249e-02
 1.42285165e-03 2.12986613e-02 8.36281356e-03 1.01598938e-04
 7.53331676e-04 4.73953280e-04 1.32101521e-03 5.77091028e-04
 2.63999652e-04 4.83630677e-04 2.30326763e-05 0.00000000e+00
 4.19594458e-06 0.00000000e+00 2.30207139e-02 1.20185043e-01
 1.68251280e-02 2.04180132e-02 7.42462852e-03 1.43613835e-03
 0.00000000e+00 9.29931000e-04 5.76700993e-04 1.56562344e-04
 7.72657279e-06 5.839029

### Predict the Testing Data and Evaluate

In [6]:
import numpy as np
Y_test_hat = clf.predict(X_encoded_test)
Accuracy = [1 for i in range(len(Y_test_hat)) if Y2.iloc[i] == Y_test_hat[i]]
Accuracy = round(float(np.sum(Accuracy))/len(Y_test_hat)*100,2)
print("Accuracy on Testing Data = %.2f%%"%Accuracy)

Accuracy on Testing Data = 83.90%


### Section 3. Train the Gradient Boosted Decision Trees Model

In [7]:
nTrees = 100
max_depth = 5
min_node_size = 5
verbose = 0
learning_rate = 0.05

from sklearn.ensemble import GradientBoostingClassifier
gbm_clf = GradientBoostingClassifier(n_estimators=nTrees, loss='deviance', learning_rate=learning_rate, max_depth=max_depth, \
                                    min_samples_leaf=min_node_size)
gbm_clf.fit(X_encoded_train, Y1)
print(gbm_clf.feature_importances_)

[5.42195003e-02 6.88439696e-05 5.35416177e-03 3.60576568e-04
 2.07578129e-03 1.17409449e-03 3.57472488e-05 5.69301701e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 5.41859106e-03
 6.74204489e-04 7.26875526e-05 0.00000000e+00 4.27474625e-04
 0.00000000e+00 1.12505730e-04 0.00000000e+00 6.10943699e-05
 0.00000000e+00 1.52853197e-04 2.70139994e-04 0.00000000e+00
 1.02637365e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.98090164e-01 8.26087089e-04 3.80540283e-01
 2.77608240e-05 3.66837707e-06 2.01797677e-05 6.98037209e-04
 1.49745083e-05 0.00000000e+00 4.34720243e-04 1.62872011e-02
 3.52132487e-04 8.62100835e-03 4.83439327e-03 1.72995050e-03
 3.08963762e-05 2.21652185e-04 4.76082668e-03 8.38525992e-05
 2.87937086e-03 1.60991405e-05 7.76673765e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.58128228e-04 7.25029414e-04
 4.48240507e-03 3.21519325e-04 2.86241179e-04 1.84133879e-04
 0.00000000e+00 5.22238971e-04 2.80660872e-04 4.30762281e-05
 7.40982153e-05 0.000000

### Predict the Testing Data and Evaluate

In [8]:
import numpy as np
Y_test_hat = gbm_clf.predict(X_encoded_test)
Accuracy = [1 for i in range(len(Y_test_hat)) if Y2.iloc[i] == Y_test_hat[i]]
Accuracy = round(float(np.sum(Accuracy))/len(Y_test_hat)*100,2)
print("Accuracy on Testing Data = %.2f%%"%Accuracy)

Accuracy on Testing Data = 87.26%
